# Assignment09
# Multi-label classification using neural networks
* 2019220177 Seungwook

## Initial Setting

### Mount Google Drive

In [16]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd /content/drive/My\ Drive/MIMP

/content/drive/My Drive/MIMP


### git Setting

In [18]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Assignment09/Assignment09.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git config --global user.email "sooonchang@gmail.com"
!git config --global user.name "SooonChang"

In [20]:
!git add .
!git commit -m "Assignment09: Make module"
!git push origin master

[master ffdd336] Assignment09: Make module
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite Assignment09/Assignment09.ipynb (96%)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 3.22 KiB | 659.00 KiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/SooonChang/MIMP
   19472cd..ffdd336  master -> master


## 0. Optimization

### Import Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

### Load Data


In [0]:
file_data = "Assignment09/mnist.csv"
handle_file = open(file_data, "r")
data = handle_file.readlines()
handle_file.close()

size_row = 28
size_col = 28

num_image = len(data)
count = 0

list_image = np.empty((num_image,size_row* size_col), dtype=float)
list_label = np.empty(num_image, dtype = int)

for line in data:
  line_data = line.split(',')
  label = line_data[0]
  im_vector = np.asfarray(line_data[1:])

  list_label[count] = label
  list_image[count,:] = im_vector
  count +=1


### define module

In [0]:
def sigmoid(x):
  return 1/ (1 + np.exp(-x))

class Sigmoid():
  
  def __init__(self):
    self.out = None
  
  def forward(self, x):
    out = sigmoid(x)
    self.out = out
    return out
  
  def backward(self, dout): # s' = (1-s)*s
    dx = dout*(1.0 - self.out) * self.out

    return dx

class Layer():
  def __init__(self,W,b):
    self.W = W
    self.b = b

    self.x = None
    self.x_shape = None
    # differntial
    self.dW = None
    self.db = None

  def forward(self, x):
    self.x_shape = x.shape
    x = x.reshape(x.shape[0],-1)
    self.x = x

    out = np.dot(self.x,self.W) + self.b

    return out
  
  def backward(self,dout):
    dx = np.dot(dout,self.W.T)
    self.dW = np.dot(self.x.T,dout)
    self.db = np.sum(dout, axis=0)

    dx = dx.reshape(*self.x_shape)
    return dx

class object_func():
  def __init__(self):
    self.loss = None
    self.h = None
    self.t = None

  def forward(self, y, t):
    self.t = t
    self.h = sigmoid(y)
    m = t.shape[0]
    self.loss = np.sum(-t*np.log(self.h +1e-7) -(1-t)*np.log(1-self.h +1e-7)) / m

    return self.loss
  
  def backward(self, dout=1):
    m = self.t.shape[0]
    dx = (self.h - self.t)/m
    
    return dx


In [0]:
class Network():

  def __init__(self, std = 0.01):
    # init weights
    self.params = {}
    self.params['W1'] = std*np.random.randn(784,196)
    self.params['b1'] = np.zeros(196)
    self.params['W2'] = std*np.random.randn(196,49)
    self.params['b2'] = np.zeros(49)
    self.params['W3'] = std*np.random.randn(49,10)
    self.params['b3'] = np.zeros(10)

    # create layers
    self.layers = OrderedDict()
    self.layers['Layer1'] = Layer(self.params['W1'], self.params['b1'])
    self.layers['Sigmoid1'] = Sigmoid()
    self.layers['Layer2'] = Layer(self.params['W2'], self.params['b2'])
    self.layers['Sigmod2'] = Sigmoid()
    self.layers['Layer3'] = Layer(self.params['W3'], self.params['b3'])

    self.lastLayer = object_func()
  
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    
    return x
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y,t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y==t)/ float(x.shape[0])
    return accuracy

  def gradient(self, x, t):
    # forward
    self.loss(x,t)

    # backward
    dout =1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    # save gradients
    grads = {}
    grads['W1'],grads['b1'] = self.layers['Layer1'].dW, self.layers['Layer1'].db
    grads['W2'],grads['b2'] = self.layers['Layer2'].dW,self.layers['Layer2'].db
    grads['W3'],grads['b3'] = self.layers['Layer3'].dW,self.layers['Layer3'].db

    return grads

In [0]:
## make label -> one-hot-encoding
num_train = len(train_lab)
train_enc = np.empty((num_train,10),dtype=int)
for i in range(num_train):
  for j in range(10):
    train_enc[i][j] = int(j==train_lab[i])

num_test = len(test_lab)
test_enc = np.empty((num_test,10),dtype=int)
for i in range(num_test):
  for j in range(10):
    test_enc[i][j] = int(j==test_lab[i])

In [0]:
train_img = list_image[:6000,:]
train_lab = list_label[:6000]
test_img = list_image[6000:,:]
test_lab = list_label[6000:]

In [0]:
network = Network()

num_iteration = 10000
lr = 0.01
train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

In [100]:
### Training ###

for i in range(num_iteration):
  grad = network.gradient(train_img, train_enc)

  for key in ('W1','b1', 'W2', 'b2', 'W3', 'b3'):
    network.params[key] -= lr*grad[key]
  
  train_loss = network.loss(train_img,train_enc)
  train_loss_list.append(loss)
  test_loss = network.loss(test_img,test_enc)
  test_loss_list.append(test_loss)

  train_acc = network.accuracy(train_img, train_enc)
  test_acc = network.accuracy(test_img,test_enc)
  train_acc_list.append(train_acc)
  test_acc_list.append(test_acc)

  if i%100 == 0:
    print("Epoch : ", i)
    print("train_loss: ",train_loss, "train_acc", train_acc)
    print("test_loss:", test_loss, "test_acc", test_acc)

Epoch :  0
train_loss:  6.658934964597058 train_acc 0.09783333333333333
test_loss: 6.658472456340207 test_acc 0.09675
Epoch :  10
train_loss:  5.160839143529189 train_acc 0.09783333333333333
test_loss: 5.16005327777092 test_acc 0.09675
Epoch :  20
train_loss:  4.29026730154317 train_acc 0.11316666666666667
test_loss: 4.288484768781414 test_acc 0.118
Epoch :  30
train_loss:  3.785681433779136 train_acc 0.11433333333333333
test_loss: 3.7834305055394375 test_acc 0.11225
Epoch :  40
train_loss:  3.5142936319276177 train_acc 0.11433333333333333
test_loss: 3.5123290147078534 test_acc 0.11225
Epoch :  50
train_loss:  3.3780489854274838 train_acc 0.11433333333333333
test_loss: 3.3767067361915366 test_acc 0.11225
Epoch :  60
train_loss:  3.311818521047198 train_acc 0.11433333333333333
test_loss: 3.3110379036059348 test_acc 0.11225
Epoch :  70
train_loss:  3.2797664995374105 train_acc 0.11433333333333333
test_loss: 3.2794238769168884 test_acc 0.11225
Epoch :  80
train_loss:  3.2641466186835757 t

KeyboardInterrupt: ignored